<a href="https://colab.research.google.com/github/zhanna-naumenko/DI-Bootcamp/blob/main/W5D3_ExercisesXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SQLAlchemy



In [2]:
!pip install flask_sqlalchemy

In [3]:
# 1. Write a Python program to create a SQLAlchemy model 'Student' with fields: 'id', 'studentname', and 'email'.
import sqlalchemy as db
from sqlalchemy import create_engine, engine

engine = db.create_engine('sqlite:///data.sqlite')
conn = engine.connect()
metadata = db.MetaData()

Student = db.Table('Student', metadata,
              db.Column('Id', db.Integer(), nullable=False),
              db.Column('StudentName', db.String(255), nullable=False),
              db.Column('Email', db.String(255))
              )
print(Student.columns.keys())

['Id', 'StudentName', 'Email']


In [4]:
query = Student.select() #SELECT * FROM Student
print(query)
metadata.create_all(engine)

SELECT "Student"."Id", "Student"."StudentName", "Student"."Email" 
FROM "Student"


In [5]:
# 2.Write a Python program that adds a new student to the 'students' table with a given id, studentname and email.

student_insert = Student.insert().values([
    {'Id': 1, 'StudentName': 'Alice', 'Email': 'alice_coul@gmail.com'},
    {'Id': 2, 'StudentName': 'Bob', 'Email': 'bob_sylli@gmail.com'},
    {'Id': 3, 'StudentName': 'Charlie', 'Email': 'charlie_main@gmail.com'},
    {'Id': 4, 'StudentName': 'Mike', 'Email': 'mike_lous@gmail.com'}
])
conn.execute(student_insert)
conn.commit()

In [6]:
exe = conn.execute(query) #executing the query
result = exe.fetchall() #extracting top 5 results
print(result)

[(1, 'Alice', 'alice_coul@gmail.com'), (2, 'Bob', 'bob_sylli@gmail.com'), (3, 'Charlie', 'charlie_main@gmail.com'), (4, 'Mike', 'mike_lous@gmail.com')]


In [7]:
# 3.Write a Python program that retrieves a student's information from the 'students' table using their id..
from sqlalchemy import text
conn = engine.connect()
student_id = 2
output = conn.execute(text(f"SELECT * FROM Student WHERE Id = {student_id}"))
print(output.fetchall())


[(2, 'Bob', 'bob_sylli@gmail.com')]


In [8]:
# 4.Write a Python program that updates a student's email in the 'students' table based on their id
from sqlalchemy import text
conn = engine.connect()
student_id = 3
output_new_email = conn.execute(text(f"UPDATE Student SET Email = 'free_range@gmail.com' WHERE Id = {student_id}"))
output = conn.execute(text(f"SELECT * FROM Student WHERE Id = {student_id}"))
print(output.fetchall())


[(3, 'Charlie', 'free_range@gmail.com')]


In [20]:
# 5.Write a Python program that deletes a student from the 'students' table by their id
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

engine = create_engine('sqlite:///testdatabase')
Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'

    id = Column(Integer, primary_key=True)
    studentname = Column(String)
    email = Column(String)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

new_student_1 = Student(id=1, studentname='John Doe', email='john@example.com')
session.add(new_student_1)

new_student_2 = Student(id=2, studentname='Anna Monte', email='anne@example.com')
session.add(new_student_2)

new_student_3 = Student(id=3, studentname='Jack Kolab', email='jack@example.com')
session.add(new_student_3)
session.commit()


student_id = 1

student = session.query(Student).filter(Student.id == student_id).first()

if student:
    session.delete(student)
    session.commit()
    print("Student deleted successfully.")
else:
    print("No student found with ID:", student_id)

result = session.query(Student).all()
for Student in result:
    print(Student.id, Student.studentname, Student.email)

session.close()

Student deleted successfully.
2 Anna Monte anne@example.com
3 Jack Kolab jack@example.com


<ipython-input-20-0d32d97c53f3>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [22]:
# 6.'i) 'item_id' as a primary key of 'items' table. ii) 'user_id' as a primary key of 'users' table.
# iii) 'order_id' as a primary key of 'orders' table and 'item_id' and 'user_id' as two foreigh keys.
# Insert some records in the 'items', 'users' and 'orders' tables'.
from flask import Flask
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()
engine = create_engine('sqlite:///shop.db', echo=True)
Base = declarative_base()

class Item(Base):
    __tablename__ = "items"

    item_id = db.Column(db.Integer, primary_key=True)
    item_name = db.Column(db.String(80), unique=True, nullable=False)
    item_price = db.Column(db.Float(precision=2), unique=False, nullable=False)
    item_quantity = db.Column(db.Integer, unique=False, nullable=False)


class Order(Base):
    __tablename__ = "orders"

    order_id = db.Column(db.Integer, primary_key=True)
    order_quantity = db.Column(db.Integer, unique=False, nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey("users.user_id"))
    item_id = db.Column(db.Integer, db.ForeignKey("items.item_id"))


class User(Base):
    __tablename__ = "users"

    user_id = db.Column(db.Integer, primary_key=True)
    user_name = db.Column(db.String(80), unique=True, nullable=False)
    user_email = db.Column(db.String(50), unique=True, nullable=False)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# some records into the items table
item1 = Item(item_name='Mango', item_price=10, item_quantity=100)
item2 = Item(item_name='Lime', item_price=20, item_quantity=150)

session.add_all([item1, item2])
session.commit()

# some records into the users table
user1 = User(user_name='John Doe', user_email='john_doe@gmail.com')
user2 = User(user_name='Jane Smith', user_email='jane_sm@hmail.com')

session.add_all([user1, user2])
session.commit()

# some records into the orders table
order1 = Order(user_id=1, item_id=1, order_quantity=5)
order2 = Order(user_id=2, item_id=2, order_quantity=10)

session.add_all([order1, order2])
session.commit()
session.close()

2024-04-29 08:05:19,377 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-22-e4cf33e7aef8>:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 08:05:19,383 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("items")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("items")


2024-04-29 08:05:19,387 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:INSERT INTO orders (order_quantity, user_id, item_id) VALUES (?, ?, ?) RETURNING order_id


2024-04-29 08:05:19,538 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] (10, 2, 2)


INFO:sqlalchemy.engine.Engine:[insertmanyvalues 2/2 (ordered; batch not supported)] (10, 2, 2)


2024-04-29 08:05:19,541 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [27]:
# 7.Write a  Python program to retrieve items with a price greater than a certain value from the 'items' table using the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop.db', echo=True)
Base = declarative_base()

class Item(Base):
    __tablename__ = "item"

    item_id = Column(Integer, primary_key=True)
    item_name = Column(String(80), unique=True, nullable=False)
    item_price = Column(Float(precision=2), nullable=False)
    item_quantity = Column(Integer, nullable=False)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


item1 = Item(item_name='Mango', item_price=10, item_quantity=100)
item2 = Item(item_name='Lime', item_price=20, item_quantity=150)
item3 = Item(item_name='Banana', item_price=15, item_quantity=160)
item4 = Item(item_name='Cofee', item_price=50, item_quantity=10)
item5 = Item(item_name='tea', item_price=70, item_quantity=15)

session.add_all([item1, item2, item3, item4, item5])
session.commit()


certain_price = 20
items_more_certain = session.query(Item).filter(Item.item_price > certain_price).all()
print("Items with a price greater than", certain_price)
for item in items_more_certain:
    print("Item ID:", item.item_id)
    print("Item Name:", item.item_name)
    print("Item Price:", item.item_price)
    print("Item Quantity:", item.item_quantity)
    print("------------------------------------")

session.close()


2024-04-29 08:21:33,780 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-27-3e90b4a013fa>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 08:21:33,791 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("item")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("item")


2024-04-29 08:21:33,797 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT item.item_id AS item_item_id, item.item_name AS item_item_name, item.item_price AS item_item_price, item.item_quantity AS item_item_quantity 
FROM item 
WHERE item.item_price > ?


2024-04-29 08:21:33,872 INFO sqlalchemy.engine.Engine [generated in 0.00236s] (20,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00236s] (20,)


Items with a price greater than 20
Item ID: 4
Item Name: Cofee
Item Price: 50.0
Item Quantity: 10
------------------------------------
Item ID: 5
Item Name: tea
Item Price: 70.0
Item Quantity: 15
------------------------------------
2024-04-29 08:21:33,876 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [29]:
# 8.Write a  Python program to update the quantity of an item in the 'Items' table based on its name using the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop1.db', echo=True)
Base = declarative_base()

class Item(Base):
    __tablename__ = "item"

    item_id = Column(Integer, primary_key=True)
    item_name = Column(String(80), unique=True, nullable=False)
    item_price = Column(Float(precision=2), nullable=False)
    item_quantity = Column(Integer, nullable=False)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


item1 = Item(item_name='Mango', item_price=10, item_quantity=100)
item2 = Item(item_name='Lime', item_price=20, item_quantity=150)
item3 = Item(item_name='Banana', item_price=15, item_quantity=160)
item4 = Item(item_name='Cofee', item_price=50, item_quantity=10)
item5 = Item(item_name='tea', item_price=70, item_quantity=15)

session.add_all([item1, item2, item3, item4, item5])
session.commit()


item_name_update = 'Lime'
new_quantity = 30
update_item = session.query(Item).filter(Item.item_name == item_name_update).first()

if update_item:
    update_item.item_quantity = new_quantity
    session.commit()
    print("Updated successfully.")
else:
    print("No item found with name:", item_name_update)


result = session.query(Item).all()
for item in result:
    print(item.item_id, item.item_name, item.item_price, item.item_quantity)

session.close()


2024-04-29 08:41:24,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-29-c8a9c9f9e23e>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 08:41:24,131 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("item")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("item")


2024-04-29 08:41:24,136 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT item.item_id AS item_item_id, item.item_name AS item_item_name, item.item_price AS item_item_price, item.item_quantity AS item_item_quantity 
FROM item


2024-04-29 08:41:24,250 INFO sqlalchemy.engine.Engine [generated in 0.00372s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00372s] ()


1 Mango 10.0 100
2 Lime 20.0 30
3 Banana 15.0 160
4 Cofee 50.0 10
5 tea 70.0 15
2024-04-29 08:41:24,254 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [4]:
# 9. Write a Python program to create a new order in the 'Order' table for a user and an item with a specified quantity using the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop2.db', echo=True)
Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    order_quantity = Column(Integer, unique=False, nullable=False)
    user_id = Column(Integer, nullable=False)
    item_id = Column(Integer, nullable=False)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


order1 = Order(order_quantity=20, user_id=10, item_id=100)
order2 = Order(order_quantity=5, user_id=11, item_id=1)
order3 = Order(order_quantity=30, user_id=12, item_id=120)
order4 = Order(order_quantity=15, user_id=19, item_id=15)
order5 = Order(order_quantity=56, user_id=16, item_id=36)

session.add_all([order1, order2, order3, order4, order5])
session.commit()


user_id = 20
item_id = 30
order_quantity = 36
new_order = Order(user_id=user_id, item_id=item_id, order_quantity=order_quantity)
session.add(new_order)
session.commit()


result = session.query(Order).all()
for order in result:
    print(order.order_id, order.order_quantity,order.user_id,order.item_id)

session.close()


2024-04-29 13:41:14,343 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-4-11a62343cf41>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 13:41:14,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("orders")


2024-04-29 13:41:14,355 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT orders.order_id AS orders_order_id, orders.order_quantity AS orders_order_quantity, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id 
FROM orders


2024-04-29 13:41:14,484 INFO sqlalchemy.engine.Engine [generated in 0.02289s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.02289s] ()


1 20 10 100
2 5 11 1
3 30 12 120
4 15 19 15
5 56 16 36
6 36 20 30
2024-04-29 13:41:14,493 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [6]:
# 10 Write a Python program to retrieve all orders for a specific user from the 'Order' table user using the SQLAlchemy model.

from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop3.db', echo=True)
Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, nullable=False)
    item_id = Column(Integer, nullable=False)
    order_quantity = Column(Integer, unique=False, nullable=False)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()


order1 = Order(user_id=10, item_id=10, order_quantity=20)
order2 = Order(user_id=12, item_id=1, order_quantity=5)
order3 = Order(user_id=12, item_id=120, order_quantity=30)
order4 = Order(user_id=19, item_id=15, order_quantity=15)
order5 = Order(user_id=16, item_id=36, order_quantity=56)

session.add_all([order1, order2, order3, order4, order5])
session.commit()

user_id = 12
all_orders = session.query(Order).filter(Order.user_id == user_id).all()

for order in all_orders:
  print(order.order_id, order.user_id,order.item_id, order.order_quantity)

session.close()

2024-04-29 13:54:20,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-6-600f1177874b>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 13:54:20,391 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("orders")


2024-04-29 13:54:20,396 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id, orders.order_quantity AS orders_order_quantity 
FROM orders 
WHERE orders.user_id = ?


2024-04-29 13:54:20,482 INFO sqlalchemy.engine.Engine [generated in 0.00238s] (12,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00238s] (12,)


2 12 1 5
3 12 120 30
2024-04-29 13:54:20,486 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [7]:
# 11 Write a  Python program to calculate and display the total cost of all orders for a given user using the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop4.db', echo=True)
Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, nullable=False)
    item_id = Column(Integer, nullable=False)
    order_quantity = Column(Integer, unique=False, nullable=False)
    price = Column(Integer, unique=False, nullable=False)
    total_cost = Column(Integer, unique=False, nullable=False)

    def __init__(self, user_id, item_id, order_quantity, price):
        self.user_id = user_id
        self.item_id = item_id
        self.order_quantity = order_quantity
        self.price = price
        self.total_cost = self.calculate_total_cost()

    def calculate_total_cost(self):
        return self.order_quantity * self.price

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

order1 = Order(user_id=10, item_id=10, order_quantity=20, price=5)
order2 = Order(user_id=12, item_id=1, order_quantity=5, price=20)
order3 = Order(user_id=12, item_id=120, order_quantity=30, price=32)
order4 = Order(user_id=19, item_id=15, order_quantity=15, price=25)
order5 = Order(user_id=16, item_id=36, order_quantity=56, price=2)

session.add_all([order1, order2, order3, order4, order5])
session.commit()


user_id=12
orders = session.query(Order).filter(Order.user_id == user_id).all()
total_cost = sum(order.total_cost for order in orders)

print(f"The total cost of all orders for user {user_id}: {total_cost}")

session.close()

2024-04-29 14:23:14,704 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-7-85aa2b6c3f8c>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 14:23:14,707 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("orders")


2024-04-29 14:23:14,712 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id, orders.order_quantity AS orders_order_quantity, orders.price AS orders_price, orders.total_cost AS orders_total_cost 
FROM orders 
WHERE orders.user_id = ?


2024-04-29 14:23:14,923 INFO sqlalchemy.engine.Engine [generated in 0.00264s] (12,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00264s] (12,)


The total cost of all orders for user 12: 1060
2024-04-29 14:23:14,926 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [10]:
# 12 Write a  Python program to list the top 2 products by sales quantity from the 'orders' table using the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func

engine = create_engine('sqlite:///new_shop6.db', echo=True)
Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, nullable=False)
    item_id = Column(Integer, nullable=False)
    order_quantity = Column(Integer, unique=False, nullable=False)
    price = Column(Integer, unique=False, nullable=False)
    total_cost = Column(Integer, unique=False, nullable=False)

    def __init__(self, user_id, item_id, order_quantity, price):
        self.user_id = user_id
        self.item_id = item_id
        self.order_quantity = order_quantity
        self.price = price
        self.total_cost = self.calculate_total_cost()

    def calculate_total_cost(self):
        return self.order_quantity * self.price

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

order1 = Order(user_id=10, item_id=10, order_quantity=20, price=5)
order2 = Order(user_id=12, item_id=1, order_quantity=5, price=20)
order3 = Order(user_id=12, item_id=120, order_quantity=30, price=32)
order4 = Order(user_id=19, item_id=15, order_quantity=15, price=25)
order5 = Order(user_id=16, item_id=36, order_quantity=56, price=2)

session.add_all([order1, order2, order3, order4, order5])
session.commit()


top_orders = (session.query(Order.item_id, func.sum(Order.order_quantity).label('total_quantity')).group_by(Order.item_id) .order_by(func.sum(Order.order_quantity).desc()).limit(2))
result = top_orders.all()

print("Top 2 products by sales:")

for order in result:
    print(order.item_id, order.total_quantity)

session.close()

2024-04-29 14:49:59,414 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-10-b010f30c628d>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 14:49:59,418 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("orders")


2024-04-29 14:49:59,422 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT orders.item_id AS orders_item_id, sum(orders.order_quantity) AS total_quantity 
FROM orders GROUP BY orders.item_id ORDER BY sum(orders.order_quantity) DESC
 LIMIT ? OFFSET ?


2024-04-29 14:49:59,520 INFO sqlalchemy.engine.Engine [generated in 0.00318s] (2, 0)


INFO:sqlalchemy.engine.Engine:[generated in 0.00318s] (2, 0)


Top 2 products by sales:
36 168
120 90
2024-04-29 14:49:59,530 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [15]:
# 13 Write a Python program to delete an order from the 'Order' table by its id. Use the SQLAlchemy model.
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///new_shop7.db', echo=True)
Base = declarative_base()

class Order(Base):
    __tablename__ = "orders"

    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, nullable=False)
    item_id = Column(Integer, nullable=False)
    order_quantity = Column(Integer, unique=False, nullable=False)
    price = Column(Integer, unique=False, nullable=False)
    total_cost = Column(Integer, unique=False, nullable=False)

    def __init__(self, user_id, item_id, order_quantity, price):
        self.user_id = user_id
        self.item_id = item_id
        self.order_quantity = order_quantity
        self.price = price
        self.total_cost = self.calculate_total_cost()

    def calculate_total_cost(self):
        return self.order_quantity * self.price

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

order1 = Order(user_id=10, item_id=10, order_quantity=20, price=5)
order2 = Order(user_id=12, item_id=1, order_quantity=5, price=20)
order3 = Order(user_id=12, item_id=120, order_quantity=30, price=32)
order4 = Order(user_id=19, item_id=15, order_quantity=15, price=25)
order5 = Order(user_id=16, item_id=36, order_quantity=56, price=2)

session.add_all([order1, order2, order3, order4, order5])
session.commit()

order_id = 2

delete_order = session.query(Order).filter(Order.order_id == order_id).first()

if delete_order:
    session.delete(delete_order)
    session.commit()
    print("Order deleted successfully.")
else:
    print("No order found with ID:", order_id)

result = session.query(Order).all()
for order in result:
    print(order.order_id, order.user_id,order.item_id, order.order_quantity, order.price, order.total_cost)

session.close()

2024-04-29 15:17:56,131 INFO sqlalchemy.engine.Engine BEGIN (implicit)


<ipython-input-15-033869fb23d5>:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-04-29 15:17:56,136 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("orders")


2024-04-29 15:17:56,140 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id, orders.order_quantity AS orders_order_quantity, orders.price AS orders_price, orders.total_cost AS orders_total_cost 
FROM orders


2024-04-29 15:17:56,259 INFO sqlalchemy.engine.Engine [generated in 0.00234s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00234s] ()


1 10 10 20 5 100
3 12 120 30 32 960
4 19 15 15 25 375
5 16 36 56 2 112
2024-04-29 15:17:56,263 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
